<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>


## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>
4. <a href="#item44">Part 3</a>

</font>
    
</div>


In [ ]:
#!pip install tensorflow numpy matplotlib Pillow wget

<a id="item41"></a>


## Download Data


Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip


Use the following cells to download the data.


In [ ]:

!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.


<a id="item42"></a>


## Part 1


In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.


You will essentially build your classifier as follows:

1. Import libraries, modules, and packages you will need. Make sure to import the _preprocess_input_ function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.


Use the following cells to create your classifier.


In [2]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from PIL import Image

In [3]:
batch_size = 100
img_width, img_height = 224, 224


In [ ]:

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)



In [5]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

model = Sequential()

model.add(vgg_model)

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
max_steps_per_epoch = 10
max_validation_steps = 10
model.fit(
    train_generator,
    steps_per_epoch=max_steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=max_validation_steps,
    epochs=10
)


In [ ]:
model.save("TrainedModel.keras")

In [ ]:
model.summary()

In [ ]:
print(f"Кількість зображень для навчання: {train_generator.samples}")

<a id="item43"></a>


## Part 2


In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


resnet_model = load_model('TrainedModel.keras')
vgg_model = load_model('TrainedModel.keras')

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

resnet_score = resnet_model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
vgg_score = vgg_model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"ResNet50 Model Accuracy: {resnet_score[1] * 100:.2f}%")
print(f"ResNet50 Model Loss: {resnet_score[0]:.4f}")

print(f"VGG16 Model Accuracy: {vgg_score[1] * 100:.2f}%")
print(f"VGG16 Model Loss: {vgg_score[0]:.4f}")


<a id="item44"></a>


## Part 3


In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>


Use the following cells to make your predictions.


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from PIL import Image

resnet_model = load_model('TrainedModel.keras')

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

predictions = resnet_model.predict(test_generator, verbose=1)

predicted_classes = np.argmax(predictions, axis=1)

class_labels = ["Negative", "Positive"]

class_indices = test_generator.class_indices
class_indices = {label: np.where(test_generator.classes == i)[0] for i, label in enumerate(class_indices)}

num_images_per_class = 5
for class_label in class_indices:
    indices = class_indices[class_label]
    random_indices = np.random.choice(indices, size=num_images_per_class, replace=False)
    for i in random_indices:
        img_path = test_generator.filepaths[i]
        img = mpimg.imread(img_path)
        plt.imshow(img)
        plt.title(f"Prediction: {class_labels[predicted_classes[i]]}")
        plt.axis('off')
        plt.show()

### Thank you for completing this lab!

This notebook was created by Alex Aklson.


This notebook is part of a course on **Coursera** called _AI Capstone Project with Deep Learning_. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).


<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
